In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv


# 0. Together

Neural Networks are great. Their flexibility (layers...connections...activation functions...and more!) allows you to build complex models that can accurately model complex relationships between predictors and outcomes. But I want to caution you: Neural Networks aren't magic. I often see people using them unnecessarily, just because they sound cool. If you're going to use NN's, make sure they're the right tool for your problem.


When building a neural network you need to think about 2 (main) things:

1. The Structure of the model (nodes/connections/activation functions)
2. The Loss Function (how do we measure how well our model is doing?)


# 1. Building a Simple NN


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Music_data.csv")
feats = ["danceability", "energy", "loudness","acousticness"]
predict = "valence"

print(df.shape)
X = df[feats]
y = df[predict]

z = StandardScaler()

X = z.fit_transform(X)

(2553, 14)


the model below has the same shape as a simple linear regression, like we talked about in lecture. It has an input layer with 4 inputs ("danceability", "energy", "loudness","acousticness"), and 1 output layer for "valence".

<img src="https://drive.google.com/uc?export=view&id=16JR3yX1gs7oC1isJAaJixNkrnZmdOxn1" width = 800px />

In [3]:
# Regression

#structure of the model
model = kb.Sequential([
    kb.layers.Dense(1, input_shape =[4]), #input
])

#how to train the model
model.compile(loss = "mean_squared_error",
              optimizer = kb.optimizers.SGD())

#fit the model (same as SKlearn)
model.fit(X,y, epochs = 100)

Epoch 1/100
2553/2553 [==============================] - 0s 51us/step - loss: 0.4963
Epoch 2/100
2553/2553 [==============================] - 0s 16us/step - loss: 0.0670
Epoch 3/100
2553/2553 [==============================] - 0s 16us/step - loss: 0.0469
Epoch 4/100
2553/2553 [==============================] - 0s 20us/step - loss: 0.0405
Epoch 5/100
2553/2553 [==============================] - 0s 15us/step - loss: 0.0377
Epoch 6/100
2553/2553 [==============================] - 0s 16us/step - loss: 0.0364
Epoch 7/100
2553/2553 [==============================] - 0s 17us/step - loss: 0.0357
Epoch 8/100
2553/2553 [==============================] - 0s 15us/step - loss: 0.0354
Epoch 9/100
2553/2553 [==============================] - 0s 15us/step - loss: 0.0352
Epoch 10/100
2553/2553 [==============================] - 0s 15us/step - loss: 0.0351
Epoch 11/100
2553/2553 [==============================] - 0s 14us/step - loss: 0.0351
Epoch 12/100
2553/2553 [==============================] - 0s 15

2553/2553 [==============================] - 0s 16us/step - loss: 0.0350
Epoch 97/100
2553/2553 [==============================] - 0s 17us/step - loss: 0.0350
Epoch 98/100
2553/2553 [==============================] - 0s 15us/step - loss: 0.0350
Epoch 99/100
2553/2553 [==============================] - 0s 14us/step - loss: 0.0350
Epoch 100/100
2553/2553 [==============================] - 0s 14us/step - loss: 0.0350


In [4]:
# build a linear regression model using LinearRegression and fitting on X and y (no need for model validation here)

lr = LinearRegression().fit(X,y)
### YOUR CODE HERE ###

In [5]:
# get weights from Neural Network
model.get_weights()


[array([[ 0.10874633],
        [ 0.09833797],
        [-0.03422379],
        [ 0.03503103]], dtype=float32),
 array([0.47110033], dtype=float32)]

In [6]:
# get the coef_ and intercept_ from your linear regression model
lr.coef_
### YOUR CODE HERE ###


array([ 0.10888319,  0.09749521, -0.03496317,  0.03511044])

### *Question*
What happens to the weights from our neural net as you **increase** the number of epochs (compare to the coefs from the linear regression model)?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" width = 200px />


### Answer

they're very similar already, and get more and more similar as you increase the epochs (which makes sense as they both have the same structure).

# 2. Parameter Bloat

Remember that a densely connected layer (`Dense()` in keras) is connected to EVERY node in the layer before and after it. The parameters can add up QUICKLY. Looking at the image of a densely connected, deep neural network below, try to calculate how many parameters (weights + bias) need to be estimates for that neural network (it's okay if you're off by a litte).

<img src="https://drive.google.com/uc?export=view&id=19mh5qaqStcZzxir6fSWHkaiEtwMiVIVT" width = 600px />

### *Question*

What do you think can happen when you have a ton of parameters and only a little data?
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" width = 200px />


### Answer

We can get overfitting. NN's are SUPER flexible, which is why you can do so many cool things with them. However, with increased flexibility/complexity comes a risk of overfitting. (however check out "Double Descent" if you're interested, it's the idea that with DEEP learning, that bias/variance trade off doesn't hold in the exact same way....)


# 3. Building a Deep Neural Net

Run the following model on the dataset `nn`. You can use `nn_test` as the test set.

In [7]:
nn = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/NN.csv")
nn_test = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/NN_test.csv")

X = nn[["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]]
y = nn[["V10"]]

X_test = nn_test[["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]]
y_test = nn_test[["V10"]]


print(nn.shape, nn_test.shape)

(100, 10) (50, 10)


Build a Model with layers with 9,7,5,3,2,1 nodes respectively. Fill in the appropriate numbers to relace the `???`s. I've done the 9 and 7 for you.

In [8]:
## DEEP MODEL
#structure of the model
model2 = kb.Sequential([
    kb.layers.Dense(7, input_shape =[9]), #input
    kb.layers.Dense(5),
    kb.layers.Dense(3),
    kb.layers.Dense(2),
    kb.layers.Dense(1) #output
])
#how to train the model
model2.compile(loss = "mean_squared_error",
              optimizer = kb.optimizers.SGD())

#fit the model (same as SKlearn)
model2.fit(X,y, epochs = 100)

Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 1.0519
Epoch 2/100
100/100 [==============================] - 0s 47us/step - loss: 0.9916
Epoch 3/100
100/100 [==============================] - 0s 47us/step - loss: 0.9589
Epoch 4/100
100/100 [==============================] - 0s 51us/step - loss: 0.9303
Epoch 5/100
100/100 [==============================] - 0s 69us/step - loss: 0.9091
Epoch 6/100
100/100 [==============================] - 0s 67us/step - loss: 0.8929
Epoch 7/100
100/100 [==============================] - 0s 90us/step - loss: 0.8794
Epoch 8/100
100/100 [==============================] - 0s 75us/step - loss: 0.8544
Epoch 9/100
100/100 [==============================] - 0s 65us/step - loss: 0.8363
Epoch 10/100
100/100 [==============================] - 0s 54us/step - loss: 0.8203
Epoch 11/100
100/100 [==============================] - 0s 71us/step - loss: 0.7991
Epoch 12/100
100/100 [==============================] - 0s 63us/step - loss: 0.7838
Ep

100/100 [==============================] - 0s 46us/step - loss: 0.3107
Epoch 99/100
100/100 [==============================] - 0s 51us/step - loss: 0.3086
Epoch 100/100
100/100 [==============================] - 0s 45us/step - loss: 0.3148


Now use `nn` and `nn_test`, and calculate the train and test MSEs.

In [10]:
### YOUR CODE HERE ###

print(mean_squared_error(model2.predict(X),y))
print(mean_squared_error(model2.predict(X_test),y_test))

0.3217318957089084
0.373792789025582


# Other Random NN Topics That are Cool
* Deep Neural Networks (NN's that have 2+ hidden layers)
* Dropout (a way to regularize NNs)
* Double Descent (You won't believe what this means for bias/variance tradedoff!!!)
* Autoencoders (NN's that do non-linear PCA)
* Generative Adversarial Networks (GANs; builds a model that can generate fake data, like faces, or paintings)
* Recurrent Neural Networks (used for time series like sentences, music, stocks...even[harry potter](https://www.digitaltrends.com/cool-tech/harry-potter-ai-story/))
* StyleGAN
* Convolutional Neural Networks (often used for images, or other spatial data)
* Shap values (a way to estimate the effect of different predictors in the NN)

Check out [this video](https://www.youtube.com/watch?v=JBlm4wnjNMY) I wrote for Crash Course about Neural Nets.